In [42]:
import csv
import pathlib

import lxml.etree as etree

In [41]:
l10n_root = pathlib.Path('/Users/apollov/work/CDDA-new/CDDA/CDDA-loc')
l10n_csv = pathlib.Path('/Users/apollov/work/CDDA-new/CDDA/Translations.csv')

In [6]:
language_dirs = [x for x in l10n_root.iterdir() if x.is_dir()]

In [9]:
xliffs = list(l10n_root.glob('*/Localized Contents/*.xliff'))
xliffs

[PosixPath('/Users/apollov/work/CDDA-new/CDDA/CDDA-loc/hu.xcloc/Localized Contents/hu.xliff'),
 PosixPath('/Users/apollov/work/CDDA-new/CDDA/CDDA-loc/en.xcloc/Localized Contents/en.xliff')]

In [46]:
def findall(tree, tag):
    return tree.findall(f'.//{{urn:oasis:names:tc:xliff:document:1.2}}{tag}')


def get_targets_from_xliff(xliff):
    with open(xliff) as f:
        tree = etree.parse(f)
    
    targets = '\n'.join([x.text for x in findall(tree, 'target')])
    return targets

print(get_targets_from_xliff(xliffs[0]))

N Toggle minimap
1
U Unload or empty wielded item
m View map
Info
7
L Move view East
0 View help
: Center view
Look
4
D
R Read
Y Manage zones
Inventory
V List all items around the player
I
W
A Apply or use wielded item
/ Advanced inventory management
v View morale
{ Toggle map memory
Misc
6
3
9
SPACE
a Apply or use item
& Craft items
_ Select martial arts style
} Toggle panel admin
+ Re-layer armor/clothing
Combat
2
SPACE
P View message log
X Peek around corners
[ View/activate mutations
= Swap inventory letters
G Grab something nearby
K Move view North
( Disassemble items
e Examine nearby terrain
) View scores
c Close door
| Wait for several minutes
> Descend stairs
D Drop item to adjacent tile
- Recraft last recipe
TAB
" Movement mode menu
J Move view South
ESC
i Open inventory
M View missions
s Toggle burst/auto mode
m Change aim mode
* Toggle snap-to-target
f Fire wielded item
5
w Wield
r Reload item
< Ascend stairs
⮐
C
d Drop item
8
H Move view West
Craft
x Look around
@ View play

In [70]:
def save_translation(xliff: pathlib.Path):
    with open(xliff) as f:
        tree = etree.parse(f)
    
    with open(l10n_csv) as f:
        translations_csv_reader = csv.reader(f)
        langs = next(translations_csv_reader)[1:]
        translation_mappings = {row[0]: dict(zip(langs, row[1:])) for row in translations_csv_reader}
    
    lang = xliff.name.split('.')[0]
    trans_units = findall(tree, 'trans-unit')
    
    for trans_unit in trans_units:
        source = findall(trans_unit, 'source')[0]
        
        try:
            translation = translation_mappings[source.text][lang]
        except KeyError:
            print(f'txn for {lang}/{source.text} not found. Skipping.')
            continue

        try:
            target = findall(trans_unit, 'target')[0]
        except IndexError:
            target = etree.SubElement(trans_unit, 'target')

        target.text = translation
        
    tree.write(str(xliff))


save_translation(xliffs[0])

In [64]:
%debug

> <ipython-input-63-2b4f3bc0b830>(15)save_translation()
     13     for trans_unit in trans_units:
     14         source = findall(trans_unit, 'source')[0]
---> 15         target = findall(trans_unit, 'target')[0]
     16         target.text = translation_mappings[source.text][lang]
     17 



ipdb>  etree.dump(trans_unit)


<trans-unit xmlns="urn:oasis:names:tc:xliff:document:1.2" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" id="Overlay UI enabled" xml:space="preserve">
        <source>Overlay UI enabled</source>
        <note>A single strings file, whose title is specified in your preferences schema. The strings files provide the localized content to display to the user for each of your preferences.</note>
      </trans-unit>
    


ipdb>  q


In [ ]:
etree.dump(trans_unit)